<a href="https://colab.research.google.com/github/kingsgameBen/MLbasic_0223/blob/main/mlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### TensorFlow 2版 （把keras融合進來了）
import keras.xx -> import tensorflow.keras.xxx

pycharm 3.8就安裝最新的, 3.7:安裝2.0.0版




In [ ]:
# 安裝資料集 - 明斯特資料集(手寫數字)
from tensorflow.keras.datasets.mnist import load_data
# ((x_train, y_train), (x_test, y_test))
# numpy array ---  m*n 的多維陣列
(x_train, y_train), (x_test, y_test) = load_data()


In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
print(x_train.shape[1]*x_train.shape[2])

(imshow)[https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.imshow.html ]


In [ ]:
import matplotlib.pyplot as plt
# 顯示數字圖片
# matplotlib colormap  關鍵參數cmap
# plt.imshow(x_train[0], cmap="PiYG")   # pink~yellowgreen
# plt.imshow(x_train[0], cmap="PiYG_r") # 反向
# plt.imshow(x_train[0], cmap="gray"
plt.imshow(x_train[0], cmap="bwr_r")

In [ ]:
import pandas as pd
pd.DataFrame(x_train[0])

In [ ]:
from tensorflow.keras.models import Sequential
# 稠密層,全連接層: 每一根神經都跟前面的輸入相連
# [ https://keras.io/zh/layers/core/ ]
from tensorflow.keras.layers import Dense
total_train_size = x_train.shape[1]*x_train.shape[2]
layers = [
        #   784 * 128 + 128(bias)
          Dense(128, activation="relu", input_dim=total_train_size),
        #   128 * 10 + 10(bias)
          Dense(10, activation="softmax")
]
model = Sequential(layers)
model.summary()
# 調整參數
# 128 * 10 = 1280 + 10(bias)